In [3]:
import torch

print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA: True
GPU: NVIDIA GeForce RTX 5080 Laptop GPU


In [4]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
      model_name="unsloth/Llama-3.2-1B-Instruct",  # Small 1B model, fast to load
      max_seq_length=2048,
      load_in_4bit=True,  # QLoRA - saves VRAM
)
print("Model loaded!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
WARNING 01-30 19:54:23 [interface.py:409] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.1. vLLM: 0.11.2.
   \\   /|    NVIDIA GeForce RTX 5080 Laptop GPU. Num GPUs = 1. Max memory: 15.92 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Model loaded!


In [13]:
messages = [{"role": "user", "content": "What is 2 + 2?"}]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
input_length = inputs.shape[1]
outputs = model.generate(inputs, max_new_tokens=500)
new_tokens = outputs[0][input_length:]
print(tokenizer.decode(new_tokens, skip_special_tokens=True))

assistant

2 + 2 = 4.


In [15]:
messages = [{"role": "user", "content": "The capital of France is"}]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")

In [23]:
outputs = model.generate(inputs, max_new_tokens=6)
new_token = outputs[0][inputs.shape[1]:]
print(outputs)
print(new_token)
print(inputs)
print(inputs.shape[1])
print("First predicted token:", tokenizer.decode(new_token))

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,    966,   4448,    220,   2366,     21,    271, 128009, 128006,
            882, 128007,    271,    791,   6864,    315,   9822,    374, 128009,
         128006,  78191, 128007,    271,    791,   6864,    315,   9822,    374,
          12366]], device='cuda:0')
tensor([  791,  6864,   315,  9822,   374, 12366], device='cuda:0')
tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,    966,   4448,    220,   2366,     21,    271, 128009, 128006,
            882, 128007,    271,    791,   6864,    315,   9822,    374, 128009,
         128006,  78191, 128007,    271]], device='cuda:0')
40
First predicted token: The capital of France is Paris


In [25]:
layer = model.model.layers[0].self_attn.q_proj
print(f"Shape: {layer.weight.shape}")
print(f"Parameters: {layer.weight.numel():,}") #quantized dimensions

Shape: torch.Size([2097152, 1])
Parameters: 2,097,152


In [26]:
print(f"Hidden size: {model.config.hidden_size}")
print(f"Num layers: {model.config.num_hidden_layers}")
print(f"Num attention heads: {model.config.num_attention_heads}") # real dimensions

Hidden size: 2048
Num layers: 16
Num attention heads: 32
